In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from utils import load_run

run = load_run('1638002792')
if run is None:
    raise Exception("Invalid run id")

og_data = run['data']
cats = og_data["word"].value_counts().keys()
print(cats)

files = list(map(lambda c: f'./dataset/{c}.ndjson', cats))
try:
    img_params=run["img_params"]
except KeyError:
    raise Exception("Unknown image params. Aborting")
print(type(img_params))





model = run['model']
pca = run['pca'] if 'pca' in run else None
scaler = run['scaler'] if 'scaler' in run else None
print(f"Done loading run. PCA {'not ' if pca is None else ''}found.")

sample = og_data.sample(1000 if len(og_data) >= 1000 else len(og_data)).reset_index(drop=True)
target = sample['word'].values.tolist()
test = sample.drop(columns=['countrycode', 'timestamp', 'recognized', 'key_id', 'drawing', 'word']).to_numpy()

print('Predicting...')
if pca is not None:
    test = scaler.transform(test)
    test = pca.transform(test)
prediction = model.predict(test)
print('Done')

from sklearn.metrics import accuracy_score
print("Scoring...")
acc_score = accuracy_score(target, prediction)
print(f"Accuracy score: {acc_score}")

/Users/cristinadoroftei/Documents/School/aai/final-project/aai-final-project
/Users/cristinadoroftei/Documents/School/aai/final-project/aai-final-project/runs/1638002792
Index(['door', 'stove', 'hockey stick', 'stitches', 'cruise ship'], dtype='object')


Exception: Unknown image params. Aborting